In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
!pip install pyvi

In [17]:
import os 
import re
import gensim
import pickle
from tqdm import tqdm
from pyvi import ViTokenizer
def read_data(folder_path):
    data = []
    labels = []
    dirs = os.listdir(folder_path)
    for path in tqdm(dirs):
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in (file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-8") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = re.sub(
                    r'(?:(?:http|https):\/\/)?([-a-zA-Z0-9.]{2,256}\.[a-z]{2,4})\b(?:\/[-a-zA-Z0-9@:%_\+.~#?&//=]*)?',
                    "", lines)
                lines = re.sub(r'<[^>]+>', '', lines)
#               Tien xu li du lieu don gian
                lines = gensim.utils.simple_preprocess(lines)
                lines = ' '.join(lines)
        
#               Tách từ tiếng Việt
                lines = ViTokenizer.tokenize(lines)
                data.append(lines)
                labels.append(path)

    return data, labels
#X_data,y_data = read_data('gdrive/MyDrive/Text Classification/Data/Train')
#pickle.dump(X_data, open('gdrive/MyDrive/Text Classification/Data_full/X_train.pkl', 'wb'))
#pickle.dump(y_data, open('gdrive/MyDrive/Text Classification/Data_full/Y_train.pkl', 'wb'))
#x_test, y_test = read_data('gdrive/MyDrive/Text Classification/Data/Test')
#pickle.dump(x_test,open('gdrive/MyDrive/Text Classification/Data_full/X_test.pkl','wb'))
#pickle.dump(y_test, open('gdrive/MyDrive/Text Classification/Data_full/Y_test.pkl', 'wb'))

In [18]:
#@title  { run: "auto" }
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from tensorflow.keras.utils import to_categorical



import sklearn.metrics as metrics
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



'''
def read_data(path):
    labels = []
    texts = []
    r = open(path, 'r', encoding='utf-8')
    sentences = r.readlines()
    for sent in sentences:
        label = sent.split()[0]
        text = sent.replace(label, '').strip()
        text = text.replace(' .','.')
        
        texts.append(text)
        labels.append(label)
    return texts,labels'''


if __name__ == '__main__':
    

    X_train = pickle.load(open('gdrive/MyDrive/Text Classification/Data_full/X_train_1.pkl','rb'))
    Y_train = pickle.load(open('gdrive/MyDrive/Text Classification/Data_full/Y_train_1.pkl', 'rb'))
    X_test = pickle.load(open('gdrive/MyDrive/Text Classification/Data_full/X_test_1.pkl', 'rb'))
    Y_test = pickle.load(open('gdrive/MyDrive/Text Classification/Data_full/Y_test_1.pkl', 'rb'))


    tfidf_vect = TfidfVectorizer(analyzer='word', max_features=100000)
    tfidf_vect.fit(X_train)
    
    pickle.dump(tfidf_vect.fit(X_train),open('gdrive/MyDrive/Text Classification/Data_full/Vect.pkl','wb'))

    X_train_tfidf = tfidf_vect.transform(X_train)
    X_test_tfidf = tfidf_vect.transform(X_test)
     #print(X_train_tfidf)
    svd = TruncatedSVD(n_components=100,random_state=1998)
    svd.fit(X_train_tfidf)
    pickle.dump(svd.fit(X_train_tfidf),open('gdrive/MyDrive/Text Classification/Data_full/Svd.pkl','wb'))
    


    X_train_tfidf_svd = svd.transform(X_train_tfidf)
    #print(X_train_tfidf_svd)
    X_test_tfidf_svd = svd.transform(X_test_tfidf)
    
    #print(y_test_n)
    encoder = LabelEncoder()
    y_train_n = encoder.fit_transform(Y_train)
    y_test_n = encoder.fit_transform(Y_test)
    #print(y_train_n)
    onehot_encoder = OneHotEncoder(sparse=False)
    y_train_encoded = y_train_n.reshape(len(y_train_n), 1)
    y_test_encoded = y_test_n.reshape(len(y_test_n), 1)
    y_train_onehot = onehot_encoder.fit_transform(y_train_encoded)
    y_test_onehot = onehot_encoder.fit_transform(y_test_encoded)
    



    
   


In [19]:
from keras import models
from keras.models import *
from keras.layers import *
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from sklearn.model_selection import train_test_split
def lstm_model(dim):
    
    input_layer = Input(shape=(dim,))
    layer = Reshape((1, dim))(input_layer)
    layer = LSTM(256, activation='relu')(layer) 
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(256, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    output_layer = Dense(12, activation='softmax')(layer)
    model = models.Model(input_layer, output_layer)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [20]:
from sklearn import metrics
call_back = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3)
def train(model, X_data, y_data, X_test, y_test, n_epochs=50):   
    #split training data and validation data
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=2019)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=64,callbacks=[call_back])
    model.evaluate(X_test,y_test)
    train_predictions = model.predict(X_train)
    val_predictions = model.predict(X_val)
    test_predictions = model.predict(X_test)
    
    val_predictions = val_predictions.argmax(axis=-1)
    test_predictions = test_predictions.argmax(axis=-1)
    train_predictions = train_predictions.argmax(axis=-1)
    
    #print("Train accuract", metrics.accuracy_score(train_predictions, y_train))
    #print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    
    #print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))
    model.save('gdrive/MyDrive/Text Classification/Text_classification_dim(300).h5')

    return history

In [28]:
def create_dnn_model():
    input_layer = Input(shape=(100,))
    layer = Dense(128, activation='relu')(input_layer)
    layer = Dense(256, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    output_layer = Dense(12, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [32]:
model = lstm_model(100)
model.summary()
history = train(model=model, X_data=X_train_tfidf_svd, y_data=y_train_onehot, X_test=X_test_tfidf_svd, y_test=y_test_onehot)


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 1, 100)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_44 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_45 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_46 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_47 (Dense)             (None, 12)                154

In [39]:
from sklearn.metrics import classification_report

model1 = lstm_model(100)
model1.load_weights('gdrive/MyDrive/Text Classification/Text_classification_dim(300).h5')
y_pred = encoder.inverse_transform((model1.predict(X_test_tfidf_svd).argmax(axis=1)))
print(classification_report(Y_test, y_pred, target_names=list(encoder.classes_)))


              precision    recall  f1-score   support

    doi_song       0.78      0.91      0.84      1000
     du_lich       0.83      0.86      0.85      1000
    giai_tri       0.92      0.92      0.92      1000
    giao_duc       0.84      0.83      0.84      1000
  kinh_doanh       0.81      0.85      0.83      1000
   oto_xemay       0.93      0.97      0.95      1000
   phap_luat       0.91      0.86      0.88      1000
      so_hoa       0.89      0.91      0.90      1000
    suc_khoe       0.84      0.89      0.86      1000
    the_gioi       0.93      0.62      0.74      1000
    the_thao       0.95      0.96      0.96      1000
     thoi_su       0.78      0.80      0.79      1000

    accuracy                           0.86     12000
   macro avg       0.87      0.86      0.86     12000
weighted avg       0.87      0.86      0.86     12000



In [31]:
model2 = create_dnn_model()
history = train(model=model, X_data=X_train_tfidf_svd, y_data=y_train_onehot, X_test=X_test_tfidf_svd, y_test=y_test_onehot)

Epoch 1/50
 448/1500 [=======>......................] - ETA: 3s - loss: 0.0232 - accuracy: 0.9897

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [24]:
import gensim
import numpy as np
from pyvi import ViTokenizer
from sklearn.preprocessing import LabelEncoder
def preprocessing_doc(doc):
    data = []
    lines = gensim.utils.simple_preprocess(doc)
    lines = ' '.join(lines)
    lines = ViTokenizer.tokenize(lines)
    data.append(lines)
    return data

model1 = lstm_model(100)
model1.load_weights('gdrive/MyDrive/Text Classification/Text_classification_dim(300).h5')
#model2 = lstm_model(150)
#model2.load_weights('gdrive/MyDrive/Text Classification/Text_classification_dim(150).h5')
#model3 = lstm_model(200)
#model3.load_weights('gdrive/MyDrive/Text Classification/Text_classification_dim(200).h5')
#model4 = lstm_model(300)
#model4.load_weights('gdrive/MyDrive/Text Classification/Text_classification_dim(300).h5')
# tf.keras.models.load_model('gdrive/MyDrive/Text Classification/Text_classification_1.h5')
testdoc = '''Chiều tối 30/6, Ban Chỉ đạo phòng, chống dịch Covid-19 huyện Long Điền tổ chức 9 điểm lấy mẫu xét nghiệm nCoV cho 31.269 người dân ở xã Phước Tỉnh sau khi địa phương này có 7 trường hợp dương tính với nCoV.

Thời gian tổ chức lấy mẫu xét nghiệm cho người dân xã này diễn ra từ chiều 30/6 đến hết ngày 2/7/2021.

Về lịch trình của ca nhiễm đầu tiên, bệnh nhân 16320 (anh N.V.T, 29 tuổi) cơ quan chức năng địa phương nhận định ca nhiễm này chưa rõ nguồn lây, có lịch trình di chuyển phức tạp, tiếp xúc nhiều người.'''


test_doc = preprocessing_doc(testdoc)
tfidf_vect = pickle.load(open('gdrive/MyDrive/Text Classification/Data_full/Vect.pkl','rb'))
#print(tfidf_vect.get_feature_names())

test_doc_tfidf = tfidf_vect.transform(test_doc)
test_doc_svd = svd.transform(test_doc_tfidf)
#print(test_doc_svd.shape)

label = np.zeros((12,),dtype='float')

#print(model.predict(test_doc_svd).argmax(axis=1))

print('Model Dim 100: ',encoder.inverse_transform((model1.predict(test_doc_svd).argmax(axis=1),)))
#print('Model Dim 150: ',encoder.inverse_transform((model2.predict(test_doc_svd).argmax(axis=1),)))
#print('Model Dim 200: ',encoder.inverse_transform((model2.predict(test_doc_svd).argmax(axis=1),)))
#print('Model Dim 300: ',encoder.inverse_transform((model4.predict(test_doc_svd).argmax(axis=1),)))


Model Dim 100:  ['thoi_su']


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
from sklearn import naive_bayes,svm,linear_model,ensemble
import xgboost
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=50):
       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        train_predictions = train_predictions.argmax(axis=-1)
     
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))
print('Naive Bayes: ')
train_model(naive_bayes.MultinomialNB(), X_train_tfidf, Y_train, X_test_tfidf, Y_test, is_neuralnet=False)
print('------------')
print('BernoulliNB Naive Bayes: ')
train_model(naive_bayes.BernoulliNB(), X_train_tfidf, Y_train, X_test_tfidf, Y_test, is_neuralnet=False)
print('------------')
print('SVC: ')
#train_model(svm.SVC(), X_train_tfidf, Y_train, X_test_tfidf, Y_test, is_neuralnet=False)
print('------------')
print('Linear Model: ')
#train_model(linear_model.LogisticRegression(), X_train_tfidf, Y_train, X_test_tfidf, Y_test, is_neuralnet=False)
print('------------')
print('RandomForestClassifier: ')
#train_model(ensemble.RandomForestClassifier(), X_train_tfidf, Y_train, X_test_tfidf, Y_test, is_neuralnet=False)
print('------------')
print('Xgboost: ')
#train_model(xgboost.XGBClassifier(), X_train_tfidf, Y_train, X_test_tfidf, Y_test, is_neuralnet=False)

Naive Bayes: 
Validation accuracy:  0.857
Test accuracy:  0.8745833333333334
------------
BernoulliNB Naive Bayes: 
Validation accuracy:  0.7922083333333333
Test accuracy:  0.7486666666666667
------------
SVC: 
------------
Linear Model: 
------------
RandomForestClassifier: 
------------
Xgboost: 


In [26]:
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.layers import *


def create_dnn_model():
    input_layer = Input(shape=(100,))
    layer = Dense(32, activation='relu')(input_layer)
    layer = Dense(64, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    output_layer = Dense(12, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier



In [27]:
classifier = create_dnn_model()
train_model(classifier=classifier, X_data=X_train_tfidf_svd, y_data=y_train_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True)

Epoch 1/50
188/188 [==============================] - 1s 4ms/step - loss: 1.9754 - accuracy: 0.4544 - val_loss: 0.5634 - val_accuracy: 0.8307
Epoch 2/50
188/188 [==============================] - 1s 3ms/step - loss: 0.5233 - accuracy: 0.8417 - val_loss: 0.4853 - val_accuracy: 0.8500
Epoch 3/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4687 - accuracy: 0.8545 - val_loss: 0.4559 - val_accuracy: 0.8549
Epoch 4/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4445 - accuracy: 0.8604 - val_loss: 0.4392 - val_accuracy: 0.8590
Epoch 5/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4290 - accuracy: 0.8635 - val_loss: 0.4313 - val_accuracy: 0.8604
Epoch 6/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4156 - accuracy: 0.8642 - val_loss: 0.4207 - val_accuracy: 0.8642
Epoch 7/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4081 - accuracy: 0.8681 - val_loss: 0.4124 - val_accuracy: 0.8657
Epoch 